# 크롤러 환경설정(경로는 바꿔주세요!!)

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
import time
import re
import pandas as pd
import json
import os, sys
import tweepy

sys.path.append(os.pardir)
options = webdriver.ChromeOptions()
options.add_argument("headless")

In [6]:
driver = webdriver.Chrome('C:/Users/chaeh/chromedriver.exe')

In [7]:
def clean_text(text):
    cleaned_text = re.sub('[^ ㄱ-ㅣ가-힣]+',' ',text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [12]:
key_path = 'C:/Users/chaeh/Desktop/crwaler/crawler/이아영/api_key.json'
with open(key_path,'r') as f:
    key = json.load(f)

# 개인정보 인증을 요청하는 Handler입니다.
auth = tweepy.OAuthHandler(key['CONSUMER_KEY'], key['CONSUMER_SECRET'])

# 인증 요청을 수행합니다.
auth.set_access_token(key['ACCESS_TOKEN_KEY'], key['ACCESS_TOKEN_SECRET'])

# twitter API를 사용하기 위한 준비입니다.
api = tweepy.API(auth,wait_on_rate_limit=True)

In [13]:
#인스타 게시물을 불러오기 위한 사전작업 단계
driver.get('https://www.instagram.com/')
p_tag = WebDriverWait(driver, timeout =5).until(EC.presence_of_element_located((By.TAG_NAME, "P")))

time.sleep(3)

#인스타그램 로그인을 위한 계정정보
email='data.campuss'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'data.2021'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF').click()
time.sleep(3)
driver.find_element_by_css_selector('button.aOOlW.HoLwm').click()
time.sleep(3)

def insta_searching(word):
    url ='https://www.instagram.com/explore/tags/' + str(word)
    return url

def click_first(driver):
    first =driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(7)
    
def next_page(driver):
    next_page = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow')
    next_page.click()
    time.sleep(7)

    
def get_content(driver):
    
# 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    tag_list =[]
# 2. 본문내용
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ''
# 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall('#[A-Za-z0-9가-힣]+', content)
    tag = ''.join(tags).replace("#"," ") # "#" 제거
    tag_data = tag.split()
    for tag_one in tag_data:
        tag_list.append(tag_one)
        
    data = [content, tag_data]
    return data

In [14]:
def insta_crawling(driver,keyword, target):
    
    result_list =[]
    url = insta_searching(keyword)
    driver.implicitly_wait(2)
    driver.get(url)
    time.sleep(2)

    click_first(driver)

    for i in range(target):
        try:
            result_list.append(get_content(driver))
            next_page(driver)
        except:
            time.sleep(2)
            next_page(driver)
    df=pd.DataFrame(result_list)
    df.columns=['본문','해시태그']
    df['본문'] = df['본문'].apply(lambda x : clean_text(x))
    df['해시태그'] = df['해시태그'].apply(lambda x : str(x))
    df['해시태그'] = df['해시태그'].apply(lambda x : clean_text(x))
    df['본문']= df['본문'] + df['해시태그']
    df.drop(['해시태그'],axis=1,inplace=True)
    df = df.drop_duplicates(subset=['본문'])
    return df

In [15]:
#트위터 게시물 가져오는 단계
def twitter_crawling(keyword):
    columns = ['본문']
    df = pd.DataFrame(columns=columns)

    max_tweets = 1000
    searched_tweets = [status for status in tweepy.Cursor(api.search, q=keyword).items(max_tweets)]

    for tweet in searched_tweets:
        tweet_json = tweet._json
        tweet_text = tweet_json['text']
        row = [tweet_text]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)

    df['본문'] = df['본문'].apply(lambda x : clean_text(x))
    df = df.drop_duplicates(subset=['본문'])
    
    return df


In [16]:
keyword = '풀무원'
target = 10

In [17]:
insta_df = insta_crawling(driver,keyword, target)

In [18]:
twitter_df = twitter_crawling(keyword)

In [19]:
sns_df = pd.concat([twitter_df, insta_df],join='outer',ignore_index=True)

In [20]:
sns_df

,본문
0,머선 회 바른 먹거리 풀무원 쳐들어갔습니다 뿌뿌 회 몸을 사리지 않는 대결 풀무원 강호동
1,점심 풀무원 통새우볶음밥
2,ㅋㅋㅋ 마음만으로도 감사 그래도 다행히 한인마트가 멀지 않은 곳에 있어서 풀무원이라...
3,ㄴㄴ 풀무원꺼 있길래 걍 집어왓엇음
4,풀무원 다논 요거톡 스타볼 개입
...,...
254,공동구매 마감된공구입니다 감사합니다 에어프라이어 교체나 구입 하실분들 주목 이번에 ...
255,뮤지엄김치간 이벤트 밥 면 우리는 매 순간 뭘 먹을지 고민하는데요 김치가 들어간 메...
256,강예빈 자두자두자두 최애과일 향긋해 매주 화요일은 여러분과 라이브방송 으로 만나는날...
257,홍길동민등산로 둘레길에 쓰레기가 많지 않아서왠지 뿌듯하지만 인증해야 하는데 줍줍할게...


In [21]:
text= []
for x in range(0,256):
    try:
        tp = sns_df['본문'][x]
        tp = tp.split('.')
    except:
        print("0")
    text.append(tp)

In [22]:
import itertools
text=list(itertools.chain(*text))

In [23]:
text

['머선 회 바른 먹거리 풀무원 쳐들어갔습니다 뿌뿌 회 몸을 사리지 않는 대결 풀무원 강호동',
 '점심 풀무원 통새우볶음밥',
 'ㅋㅋㅋ 마음만으로도 감사 그래도 다행히 한인마트가 멀지 않은 곳에 있어서 풀무원이라든지 씨제이라든지 고런 회사서 나온 냉면으로 버티고 있는데 ㅠㅠㅠ 맛있는 평냉 평냉이 넘나 생각나네 ㅠㅠㅠ 이안이는 평냉 좋아하나',
 'ㄴㄴ 풀무원꺼 있길래 걍 집어왓엇음',
 '풀무원 다논 요거톡 스타볼 개입',
 '어제 풀무원 마파두부 해먹었는데 그게 일본거보다 맛있음이다 미친 놈아',
 '풀무원 매콤면떡볶이 드세요 저 이거로 로제해먹는데 존맛입니다 우유 치즈 장 후추약간 넣어서 먹으면 시중에서파는 로제맛이랑 비슷함니다 개정도해먹으면 양적당해요 개는부족함 제입맛에는 배떡 로',
 '머선 풀무원편 시청이벤트 추가당첨 민호 덕분에 풀무원 득템',
 '헐 풀무원 이벤트 당첨됐다 민호야 잘 먹을게',
 '헉 머선 풀무원편 시청한거 응모했는데 당첨됐다 민호덕에 만두먹겠다 샤이니 입덕하고 돈은 털렸지만 물질은 풍요롭네',
 '의 트친비는 풀무원 입니다 지불 방식은 알아서 해 드리며 계약자가 러브라이버일 시 코스프레 해 주기의 특전이 있습니다',
 '풀무원 잇츠 프로틴 맛있다',
 '요즘 풀무원 두부면에 꽂혀서 열심히 먹는 중',
 '풀무원 다논 위솔루션 요구르트',
 '자양강장제 비타민 홍삼 정관장 원동력 풀무원',
 '코코아의 트친비는 풀무원 입니다 지불 방식은 이미 당신이 잘 알고 있으며 학점이 이 안 나왔을 시 트친비 다음 달로 이월시켜주기의 특전이 있습니다 오 풀무원',
 '풀무원녹즙 브레드이발소 과채주스 유기농키즈프리 톡톡로니 키즈프리 유기농주스 풀무원 키즈프리취향저격이벤트 맘스다이어리',
 '판타지아 대면 아이엠 창균 민혁 대뜸 팬싸가서 요거톡파인 나와 비요뜨파인 창균 민혁이랑 토론 아닌 토론하고옴 풀무원 몬스타엑스한테 연락 꼬옥 주길',
 '풀무원 그릭요거트 너무 맛있다 당의 맛 풀무원 매수하고싶어',
 '풀무원은 해외 수출 두부에 토후 말고

In [25]:
import numpy as np

df = pd.DataFrame(sns_df,columns=['본문'])

In [26]:
drop_index = df[df['본문']==""].index
df.drop(drop_index,inplace=True)

In [27]:
from eunjeon import Mecab
tokenizer = Mecab()
pos_temp = {}
pos_list = []

for i in df['본문']:
    temp_str = ''
    try:
        for j in range(len(tokenizer.pos(i))):
            pos_temp[tokenizer.pos(i)[j][1]]+=1
            temp_str+=tokenizer.pos(i)[j][1]+"/"
    except:
        for j in range(len(tokenizer.pos(i))):
            pos_temp[tokenizer.pos(i)[j][1]]=1
            temp_str+=tokenizer.pos(i)[j][1]+"/"
    pos_list.append(temp_str)

In [28]:
pos_list

['NP/JKB+JX/MAG/VA+ETM/NNG/NNP/VV+EP/EC/VX/VX/NNBC/NNG/JKO/VV/EC/VX/ETM/NNG/NNP/NNP/',
 'NNG/NNP/NNG/NNG/NNG/',
 'IC/NNG/JX/JKB/JX/NNG/MAJ/MAG/NNG/NNG/JKS/VA/EC/VX/ETM/NNG/JKB/VV/EC/NNP/JC/NNP/JC/MM/NNG/JKB/VV+ETM/NNG/JKB/VV/EC/VX/EC/UNKNOWN/VA/ETM/NNG/XPN/NNG/NNG/JKS/VV/EC/NNG/NP/XSN/UNKNOWN/JKS/MAG/VV/ETM/NNG/NNG/VA/EC/NR/',
 'NNG/NNG/NNP/NNG/NNG/NNP/VV+EC/VX/EC/UNKNOWN/VV/EC/VV+EP/EP/ETN/',
 'NNP/NNP/EC+VX+ETM/NP/MAG/NNG/VV+ETM/NNG/',
 'MAG/NNP/NNG/MAG/NNP/NNG/XSV+EC/VV/EP/EC/NP+JKS/NNP/NNB/JKB/VA/ETN/JKS/MAG/VV+ETM/NNB/EC/',
 'NNP/NNP/XR/EC/NNG/VV+EP+EF/IC/NP/JKB/NNG/XSV+EC/VV/EC/NNP/NNG/VCP+EF/NNG/NNG/NNG/NNG/MAG/VV/EC/VV/EC/NNG/JKB/VV/ETM/NNG/NNG/JC/XR/XSA/EC/MAG/NNG/NNG/XSV+EC/VV/EC/MM/XR/XSA+EF/VV/ETM/NNG/XSV+ETN/NP+JKG+NNG+JKB/JX/NNG/NNG/JKB/',
 'NP/JKB+JX/NNP/NNB/NNG/NNG/NNG/NNG/NNG/NNG/JKB/NNP/NNG/',
 'IC/NNP/NNG/NNG/IC/NNP/NNG/NNG/XSV+EP/EC/NNG/VCP+EF/MAG/VV/ETM/NNB+JKS/',
 'IC/NP/JKB+JX/NNP/NNB/NNG/IC/NP/JKB+JX/NNP/NNB/NNG/XSA+ETM/NNB/NNG/XSV+EP/EC/NNG/XSV+EP/EC/NNG/NNG/JK